In [1]:
import pandas as pd

In [2]:
import requests

In [3]:
import datetime

In [4]:
now  = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

In [5]:
url1 = "https://www.worldometers.info/world-population/population-by-country/"

In [6]:
url2 = "https://www.worldometers.info/geography/countries-of-the-world/"

In [7]:
url_ico_code = "https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv"

In [8]:
xl_writer = pd.ExcelWriter(f"world_demographic_data_{now}.xlsx")

In [9]:
def get_demographic_data(url):
    res = requests.get(url)
    if res.status_code != 200:
        raise Exception("Can't dowload data")
    else:
        data = pd.read_html(res.text)
        df = data[0]
        if "#" in df.columns:
            df.drop(columns="#", inplace=True)
        return df


In [10]:
df1 = get_demographic_data(url1)

In [11]:
df2 = get_demographic_data(url2)

In [12]:
df1.to_excel(xl_writer, "df1")

In [13]:
df2.to_excel(xl_writer, "df2")

In [14]:
df1.columns

Index(['Country (or dependency)', 'Population (2020)', 'Yearly Change',
       'Net Change', 'Density (P/Km²)', 'Land Area (Km²)', 'Migrants (net)',
       'Fert. Rate', 'Med. Age', 'Urban Pop %', 'World Share'],
      dtype='object')

In [15]:
df1.columns = [
    'Country', 
    'Population', 
    'Yearly Change',
    'Net Change', 
    'Density', 
    'Land Area', 
    'Migrants (net)',
    'Fert. Rate', 
    'Med. Age', 
    'Urban Pop %', 
    'World Share'
]

In [16]:
df2.columns

Index(['Country', 'Population(2020)', 'Region'], dtype='object')

In [17]:
df2.drop(columns='Population(2020)', inplace=True)

In [18]:
df3 = pd.merge(df1, df2, on="Country", how="outer")

In [19]:
df3.to_excel(xl_writer, "df3")

In [20]:
df3.head()

Country  Population Yearly Change  Net Change  Density  Land Area  \
0          China  1439323776        0.39 %     5540090      153    9388211   
1          India  1380004385        0.99 %    13586631      464    2973190   
2  United States   331002651        0.59 %     1937734       36    9147420   
3      Indonesia   273523615        1.07 %     2898047      151    1811570   
4       Pakistan   220892340        2.00 %     4327022      287     770880   

   Migrants (net) Fert. Rate Med. Age Urban Pop % World Share         Region  
0       -348399.0        1.7       38        61 %     18.47 %           Asia  
1       -532687.0        2.2       28        35 %     17.70 %           Asia  
2        954806.0        1.8       38        83 %      4.25 %  North America  
3        -98955.0        2.3       30        56 %      3.51 %           Asia  
4       -233379.0        3.6       23        35 %      2.83 %           Asia

In [21]:
df4 = pd.read_csv(url_ico_code)
df4.loc[df4["name"] == "Namibia", "alpha-2"] = "NA"
df4[df4["name"] == "Namibia"]

name alpha-2 alpha-3  country-code     iso_3166-2  region  \
153  Namibia      NA     NAM           516  ISO 3166-2:NA  Africa   

             sub-region intermediate-region  region-code  sub-region-code  \
153  Sub-Saharan Africa     Southern Africa          2.0            202.0   

     intermediate-region-code  
153                      18.0

In [22]:
df4.head()

name alpha-2 alpha-3  country-code     iso_3166-2   region  \
0     Afghanistan      AF     AFG             4  ISO 3166-2:AF     Asia   
1   Åland Islands      AX     ALA           248  ISO 3166-2:AX   Europe   
2         Albania      AL     ALB             8  ISO 3166-2:AL   Europe   
3         Algeria      DZ     DZA            12  ISO 3166-2:DZ   Africa   
4  American Samoa      AS     ASM            16  ISO 3166-2:AS  Oceania   

        sub-region intermediate-region  region-code  sub-region-code  \
0    Southern Asia                 NaN        142.0             34.0   
1  Northern Europe                 NaN        150.0            154.0   
2  Southern Europe                 NaN        150.0             39.0   
3  Northern Africa                 NaN          2.0             15.0   
4        Polynesia                 NaN          9.0             61.0   

   intermediate-region-code  
0                       NaN  
1                       NaN  
2                       NaN  
3                       NaN  
4                       NaN

In [23]:
df4.to_excel(xl_writer, "df4")

In [24]:
iso_code = pd.read_csv("country_name_to_iso_code.csv")
iso_code.loc[iso_code["Country"] == "Namibia", "alpha-2"] = "NA"
iso_code[iso_code["Country"] == "Namibia"]

Unnamed: 0  Country alpha-2
150         152  Namibia      NA

In [25]:
df5 = pd.merge(df1, iso_code, how="outer", on="Country")

In [26]:
df5.to_excel(xl_writer, "df5")

In [27]:
df5

Country    Population Yearly Change  Net Change  \
0                               China  1.439324e+09        0.39 %   5540090.0   
1                               India  1.380004e+09        0.99 %  13586631.0   
2                       United States  3.310027e+08        0.59 %   1937734.0   
3                           Indonesia  2.735236e+08        1.07 %   2898047.0   
4                            Pakistan  2.208923e+08        2.00 %   4327022.0   
..                                ...           ...           ...         ...   
243  Saint Vincent and the Grenadines           NaN           NaN         NaN   
244             Sao Tome and Principe           NaN           NaN         NaN   
245                           Taiwan*           NaN           NaN         NaN   
246                                US           NaN           NaN         NaN   
247                West Bank and Gaza           NaN           NaN         NaN   

     Density  Land Area  Migrants (net) Fert. Rate Med. Age Urban Pop %  \
0      153.0  9388211.0       -348399.0        1.7       38        61 %   
1      464.0  2973190.0       -532687.0        2.2       28        35 %   
2       36.0  9147420.0        954806.0        1.8       38        83 %   
3      151.0  1811570.0        -98955.0        2.3       30        56 %   
4      287.0   770880.0       -233379.0        3.6       23        35 %   
..       ...        ...             ...        ...      ...         ...   
243      NaN        NaN             NaN        NaN      NaN         NaN   
244      NaN        NaN             NaN        NaN      NaN         NaN   
245      NaN        NaN             NaN        NaN      NaN         NaN   
246      NaN        NaN             NaN        NaN      NaN         NaN   
247      NaN        NaN             NaN        NaN      NaN         NaN   

    World Share  Unnamed: 0 alpha-2  
0       18.47 %          44      CN  
1       17.70 %         102      IN  
2        4.25 %         237      US  
3        3.51 %         103      ID  
4        2.83 %         167      PK  
..          ...         ...     ...  
243         NaN         189      VC  
244         NaN         193      ST  
245         NaN         218      TW  
246         NaN         239      US  
247         NaN         245      PS  

[248 rows x 13 columns]

In [28]:
df6 = pd.merge(df5, df4, how="inner", on="alpha-2")

In [29]:
df6.to_excel(xl_writer, "df6")

In [30]:
countries_csse = pd.DataFrame(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark',
       'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic',
       'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea',
       'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France',
       'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece',
       'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana',
       'Haiti', 'Holy See', 'Honduras', 'Hungary', 'Iceland', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy',
       'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya',
       'Korea, South', 'Kosovo', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Latvia',
       'Lebanon', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania',
       'Luxembourg', 'MS Zaandam', 'Madagascar', 'Malawi', 'Malaysia',
       'Maldives', 'Mali', 'Malta', 'Mauritania', 'Mauritius', 'Mexico',
       'Moldova', 'Monaco', 'Mongolia', 'Montenegro', 'Morocco',
       'Mozambique', 'Namibia', 'Nepal', 'Netherlands', 'New Zealand',
       'Nicaragua', 'Niger', 'Nigeria', 'North Macedonia', 'Norway',
       'Oman', 'Pakistan', 'Panama', 'Papua New Guinea', 'Paraguay',
       'Peru', 'Philippines', 'Poland', 'Portugal', 'Qatar', 'Romania',
       'Russia', 'Rwanda', 'Saint Kitts and Nevis', 'Saint Lucia',
       'Saint Vincent and the Grenadines', 'San Marino',
       'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia',
       'Seychelles', 'Sierra Leone', 'Singapore', 'Slovakia', 'Slovenia',
       'Somalia', 'South Africa', 'South Sudan', 'Spain', 'Sri Lanka',
       'Sudan', 'Suriname', 'Sweden', 'Switzerland', 'Syria', 'Taiwan*',
       'Tanzania', 'Thailand', 'Timor-Leste', 'Togo',
       'Trinidad and Tobago', 'Tunisia', 'Turkey', 'US', 'Uganda',
       'Ukraine', 'United Arab Emirates', 'United Kingdom', 'Uruguay',
       'Uzbekistan', 'Venezuela', 'Vietnam', 'West Bank and Gaza',
       'Western Sahara', 'Yemen', 'Zambia', 'Zimbabwe'])

In [31]:
countries_csse.columns = ["Country", ]

In [32]:
df7 = pd.merge(countries_csse, iso_code, how="outer", on="Country")

In [33]:
df7.to_excel(xl_writer, "df7")

In [34]:
xl_writer.close()